In [ ]:
import pandas as pd
import numpy as np
import telecontrol_parser as tp
import timeseries_functions as tf
from pandas.plotting import lag_plot
from pandas.plotting import autocorrelation_plot
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf

In [ ]:
    datasets = tf.tsdf_standard_run(
        locatie='parkhaven',
        threshold=24,
        interpolate=12
    )


In [ ]:
dataset = datasets[3]
series = dataset[dataset.columns[3]]
plt.figure()
lag_plot(series)

In [ ]:
plt.figure()
autocorrelation_plot(series)

In [ ]:
plot_acf(series, lags= 288);

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller

In [ ]:
print(adfuller(series))

In [ ]:
plt.rcParams.update({'figure.figsize':(9,7), 'figure.dpi':120})

# Import data
df = series

# Original Series
fig, axes = plt.subplots(3, 2)
axes[0, 0].plot(df); axes[0, 0].set_title('Original Series')
plot_acf(df, ax=axes[0, 1])

# 1st Differencing
axes[1, 0].plot(df.diff()); axes[1, 0].set_title('1st Order Differencing')
plot_acf(df.diff().dropna(), ax=axes[1, 1])

# 2nd Differencing
axes[2, 0].plot(df.diff().diff()); axes[2, 0].set_title('2nd Order Differencing')
plot_acf(df.diff().diff().dropna(), ax=axes[2, 1])

plt.show()

In [ ]:
from pmdarima.arima.utils import ndiffs
y = series

## Adf Test
print(ndiffs(y, test='adf'))

# KPSS test
print(ndiffs(y, test='kpss'))

# PP test:
print(ndiffs(y, test='pp'))
# d = 1 !

In [ ]:
# PACF plot of 1st differenced series
plt.rcParams.update({'figure.figsize':(9,3), 'figure.dpi':120})

fig, axes = plt.subplots(1, 2)
axes[0].plot(series.diff()); axes[0].set_title('1st Differencing')
axes[1].set(ylim=(0,0.25))
plot_pacf(series.diff().dropna(), ax=axes[1])

plt.show()
# p = 5

In [ ]:
plt.rcParams.update({'figure.figsize':(9,3), 'figure.dpi':120})

# Import data


fig, axes = plt.subplots(1, 2)
axes[0].plot(series.diff()); axes[0].set_title('1st Differencing')
axes[1].set(ylim=(0,1.2))
plot_acf(series.diff().dropna(), ax=axes[1])

plt.show()
# q = 1

In [ ]:
model = ARIMA(series,order = (5,1,1))
res = model.fit()
print(res.summary())
res.plot_diagnostics();


In [ ]:
train_series = series[:-36]
test_series = series[-36:]
model = ARIMA(train_series,order = (6,2,6))
res = model.fit()
print(res.summary())
res.plot_diagnostics();
# STOPPED RUN 144,2,144 AT 70 MINS, ARIMA TOO SLOW FOR LARGE ORDER/ do not use

In [ ]:
res.forecast(36).plot()
test_series.plot()

  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [105 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-3.10
      creating build\lib.win-amd64-3.10\matrixprofile
      copying matrixprofile\analyze.py -> build\lib.win-amd64-3.10\matrixprofile
      copying matrixprofile\compute.py -> build\lib.win-amd64-3.10\matrixprofile
      copying matrixprofile\core.py -> build\lib.win-amd64-3.10\matrixprofile
      copying matrixprofile\discover.py -> build\lib.win-amd64-3.10\matrixprofile
      copying matrixprofile\exceptions.py -> build\lib.win-amd64-3.10\matrixprofile
      copying matrixprofile\preprocess.py -> build\lib.win-amd64-3.10\matrixprofile
      copying matrixprofile\transform.py -> build\lib.win-amd64-3.10\matrixprofile
      copying matrixprofile\utils.py -> build\lib.win-amd64-3.10\matrixprofile
      copying matrix


  Using cached tsfresh-0.19.0-py2.py3-none-any.whl (97 kB)
  Using cached matrixprofile-1.1.10.tar.gz (331 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached distributed-2022.7.0-py3-none-any.whl (885 kB)
  Running setup.py clean for matrixprofile
Failed to build matrixprofile
  Running setup.py install for matrixprofile: started
  Running setup.py install for matrixprofile: finished with status 'error'
